## Import Libraries

In [ ]:
!pip install transformers
!pip install tensorflow-text==2.9.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 80.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 108.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 70.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.8/511.8 MB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 KB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 82.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 105.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.7/438.7 KB 42.4 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.11.0
    Un

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
from tensorflow.keras.layers import Input, GRU, Dense, Embedding, Bidirectional, TimeDistributed   #layers required for network
from tensorflow.keras.layers import Layer, Conv1D, Softmax, Concatenate ,Dropout, MaxPool1D        #layers required for network
from tensorflow.keras.backend import expand_dims, tile, concatenate, shape, batch_dot, squeeze     #functions required for network
import tensorflow.keras.backend as K                                                               #to build metric
from tensorflow.keras.models import Model                                                          #to build model
from tensorflow.keras.callbacks import TensorBoard                                                 #tensorboard
import tensorflow as tf                                                                            #other functions
from tqdm import tqdm                                                                              #track progress
import numpy as np                                                                                 #for numpy operations
import pickle                                                                            #loading tokenizers
import tensorflow_hub as hub
import tensorflow_text as text
from transformers import BertTokenizer
import math
from numpy import save, load
import tensorflow.nn as nn

In [ ]:
print(tf.executing_eagerly())
print(tf.__version__)

True
2.9.3


In [ ]:
len(tf.config.list_physical_devices('GPU'))

1

## Loading all the Required Variables from disk

In [ ]:
# y_start_train_a = load("/content/y_start_train.npy")
# y_end_train_a = load("/content/y_end_train.npy")

# train_input_id_a = load("/content/ids_train (1).npy")
# train_input_mask_a = load("/content/mask_train (1).npy")
# train_input_type_a = load("/content/type_train (1).npy")


In [ ]:
# Loading Data
y_start_train = load("/content/gdrive/MyDrive/Colab Notebooks/InterIIT Dataset/y_start_train.npy")
y_end_train = load("/content/gdrive/MyDrive/Colab Notebooks/InterIIT Dataset/y_end_train.npy")
y_end_test = load("/content/gdrive/MyDrive/Colab Notebooks/InterIIT Dataset/y_end_test.npy")
y_start_test = load("/content/gdrive/MyDrive/Colab Notebooks/InterIIT Dataset/y_start_test.npy")

train_input_id = load("/content/gdrive/MyDrive/Colab Notebooks/InterIIT Dataset/ids_train.npy")
train_input_mask = load("/content/gdrive/MyDrive/Colab Notebooks/InterIIT Dataset/mask_train.npy")
train_input_type = load("/content/gdrive/MyDrive/Colab Notebooks/InterIIT Dataset/type_train.npy")

test_input_id = load("/content/gdrive/MyDrive/Colab Notebooks/InterIIT Dataset/ids_test.npy")
test_input_mask = load("/content/gdrive/MyDrive/Colab Notebooks/InterIIT Dataset/mask_test.npy")
test_input_type = load("/content/gdrive/MyDrive/Colab Notebooks/InterIIT Dataset/type_test.npy")

In [ ]:
y_start_train.shape

(60016, 461)

In [ ]:
train_input_id = tf.convert_to_tensor(train_input_id,dtype=tf.int32)
train_input_mask = tf.convert_to_tensor(train_input_mask,dtype=tf.int32)
train_input_type = tf.convert_to_tensor(train_input_type,dtype=tf.int32)

In [ ]:
train_input_type.shape

TensorShape([60016, 512])

# Making Layers

## Bert Layer

In [ ]:
class Bert(Layer):

    def __init__(self):
        super().__init__()
        self.bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")        
    def build(self, input_shape):
        self.built = True

    def call(self, inputs):
        input_id,input_mask,input_type = inputs
        outputs = self.bert_encoder({'input_mask': input_mask,'input_type_ids': input_type,'input_word_ids': input_id},False)
        context_features = outputs['sequence_output'][:, 0:461,:]
        question_features = tf.concat((tf.expand_dims(outputs['sequence_output'][:, 0, :],axis=1), outputs['sequence_output'][:, 461:, :]), axis=1)
        return question_features,context_features

## Attention Layer

In [ ]:
class AttFlow(Layer):

    def __init__(self, feature_dimension):
        super().__init__()
        
        #output shape: 1, input shape: 3*Dim
        self.weight = Dense(1,input_shape=(3*feature_dimension,), activation=None, use_bias=False)
    
    def build(self, input_shape):
        self.built = True

    def call(self, inputs):
        context_features, question_features = inputs
        # Construct a similarity matrix
        batch_size = 16          # N
        feature_dimension = 768   # d
        length_context = 461     # T
        length_quesiton = 52    # J

        # if batch_size == None:
        #     batch_size = 0

        shape = (batch_size, length_context, length_quesiton,feature_dimension)    # (N,T,J,d)
        
        #INITIALLY context_features: (N,T,d), question features: (N,J,d)

        context_features_expanded = tf.expand_dims(context_features,2)                  # (N,T,1,d)
        # print(context_features_expanded.shape)
        broadcast_shape = tf.where([True, False, False, False],tf.shape(context_features_expanded), [batch_size, length_context, length_quesiton,feature_dimension])
        context_features_expanded = tf.broadcast_to(context_features_expanded,broadcast_shape)    # (N,T,J,d)
        
        question_features_expanded = tf.expand_dims(question_features,1)                # (N,1,J,d)
        broadcast_shape = tf.where([True, False, False, False],tf.shape(question_features_expanded), [batch_size, length_context, length_quesiton,feature_dimension])
        question_features_expanded = tf.broadcast_to(question_features_expanded,broadcast_shape)  # (N,T,J,d)

        entrywise_prod = tf.math.multiply(context_features_expanded, question_features_expanded)   # (N,T,J,d)
        
        concat_feature = tf.concat( [context_features_expanded, question_features_expanded, entrywise_prod], axis=-1)  # (N,T,J,3d)
        
        broadcast_shape = tf.where([True, False, False],[16, length_context, length_quesiton], [batch_size, length_context, length_quesiton])
        similarity = tf.reshape(self.weight(concat_feature) ,broadcast_shape)  # (N,T,J)

        # Context2Question attention
        weight_c2q = nn.softmax(similarity, axis=-1) # (N,T,J)

        # (N,T,J) * (N,J,d) -> (N,T,d)
        c2q = tf.matmul(weight_c2q, question_features)

        # Question2Context attention
        weight_q2c = nn.softmax( tf.reduce_max(similarity, axis=[2]), axis=-1)  # (N,T)

        # (N,1,T) * (N,T,d) -> (N,1,d)
        q2c = tf.matmul(tf.expand_dims(weight_q2c, 1), context_features)

        q2c = tf.tile(q2c, [1, length_context,1])   # (N,T,d)

        return c2q, q2c

### Modelling Layer

In [ ]:
class modelling_input_layer(Layer):
    
    def __init__(self):
        
        super(modelling_input_layer, self).__init__()
        
    def build(self, input_shape):
        self.built = True

    def call(self, inputs):
        
        H, c2q, q2c = inputs
        G = concatenate([H, c2q, (H*c2q), (H*q2c)], axis=2)
        
        return G

In [ ]:
class modelling_layer(Layer):
    
    def __init__(self, output_dim):
        
        super(modelling_layer, self).__init__()
        self.output_dim = output_dim
        self.modelling1 = Bidirectional(GRU(self.output_dim, return_sequences=True, dropout=0.2))
        self.modelling2 = Bidirectional(GRU(self.output_dim, return_sequences=True, dropout=0.2))
        
    def build(self, input_shape):
        self.built = True

    def call(self, inputs):
        return self.modelling2(self.modelling1(inputs))
    
    def get_config(self):

        config = super().get_config().copy()
        config.update({
            'output_dim': self.output_dim,
        })
        
        return config

### Output Layer

In [ ]:
class input_to_start(Layer):
    
    def __init__(self):
        
        super(input_to_start, self).__init__()
        
    def build(self, input_shape):
        self.built = True

    def call(self, inputs):
        
        G, M = inputs
        GM = concatenate([G, M], axis=2)
        
        return GM

In [ ]:
class output_start(Layer):
    
    def __init__(self):
        
        super(output_start, self).__init__()
        self.dense = Dense(1, activation = "linear", kernel_initializer=tf.keras.initializers.glorot_uniform(seed=35))
        self.dropout = Dropout(0.2)
        
    def build(self, input_shape):
        self.built = True

    def call(self, inputs):
        
        GM = inputs
        start = self.dense(GM)
        start = self.dropout(start)
        p1 = tf.nn.softmax(squeeze(start, axis=2))
        
        return p1

In [ ]:
class input_to_end(Layer):
    
    def __init__(self, output_dim):
        
        super(input_to_end, self).__init__()
        self.output_dim = output_dim
        self.end = Bidirectional(GRU(self.output_dim, return_sequences=True, dropout=0.2, kernel_initializer=tf.keras.initializers.glorot_uniform(seed=5)))
        
    def build(self, input_shape):
        self.built = True

    def call(self, inputs):
        
        G, M = inputs
        M2 = self.end(M)
        GM2 = concatenate([G, M2], axis=2)
        return GM2
    
    def get_config(self):

        config = super().get_config().copy()
        config.update({
            'output_dim': self.output_dim,
        })
        
        return config

In [ ]:
class output_end(Layer):
    
    def __init__(self):
        
        super(output_end, self).__init__()
        self.dense = Dense(1, activation = "linear", kernel_initializer=tf.keras.initializers.glorot_uniform(seed=85))
        self.dropout = Dropout(0.2)
        
    def build(self, input_shape):
        self.built = True

    def call(self, inputs):
        
        GM2 = inputs
        end = self.dense(GM2)
        end = self.dropout(end)
        p2 = tf.nn.softmax(squeeze(end, axis=2))
        
        return p2

## Define Model

In [ ]:
def bidaf_model(hidden_size,  n_filters, filter_width):
    """Function that build the BiDAF model using the custom class layers"""
    
    #inputs
    input_id = Input(shape=(512),name = 'Ids',dtype=tf.int32)
    input_mask = Input(shape=(512),name = 'mask',dtype=tf.int32)
    input_type = Input(shape=(512),name = 'Type',dtype=tf.int32)
    # question_words = Input(shape=(question_timesteps,), name = 'question_word_tokens')
    # context_words = Input(shape=(context_timesteps,), name = 'context_word_tokens')    
    # question_chars = Input(shape=(question_timesteps,char_max,), name = 'question_char_tokens')
    # context_chars = Input(shape=(context_timesteps,char_max,), name = 'context_char_tokens')

    # #word embedding layer
    # question_word_embedded, context_word_embedded = word_embedding_layer(len(word_tokenizer)+1, hidden_size, question_words.shape[-1])([question_words, context_words])
            
    # #character embedding layer
    # question_char_embedded, context_char_embedded = char_embedding_layer(len(char_tokenizer.word_index)+1, char_vocab, question_chars.shape[-1])([question_chars, context_chars])
    
    # #character CNN
    # question_char_embedded, context_char_embedded  = char_cnn_layer(n_filters, filter_width)([question_char_embedded, context_char_embedded])
    
    # #highway layer
    # context, question = highway_input_layer()([question_word_embedded, context_word_embedded, question_char_embedded, context_char_embedded])
    # question_blendrep = highway_layer("question_highway")(question)
    # context_blendrep = highway_layer("context_highway")(context)

    # #contextual layer
    # question_contextual = contextual_layer(hidden_size, "question_contextual")(question_blendrep)
    # context_contextual = contextual_layer(hidden_size, "context_contextual")(context_blendrep)
    
    
    Question_features,Context_features = Bert() ([input_id,input_mask,input_type])

    #attention layer
    c2q,q2c = AttFlow(768)([Context_features, Question_features])
    # attention_in = attention_input_layer()([context_contextual, question_contextual])
    # attention_out = attention_layer()(attention_in)
    # c2q, q2c = c2q_q2c_layer()([attention_out, context_contextual, question_contextual])
    
    #modelling layer
    G = modelling_input_layer()([Context_features, c2q, q2c])
    M = modelling_layer(hidden_size)(G)
    
    #output layers
    GM = input_to_start()([G,M])
    start = output_start()(GM)
    GM2 = input_to_end(hidden_size)([G,M])
    end = output_end()(GM2)

    model = Model(inputs=[input_id,input_mask,input_type], outputs=[start, end], name="bidaf")
        
    return model

### Build Model

In [ ]:
#defining various parameters of the model and building it

#input timesteps
# question_timesteps = question_max
# context_timesteps = context_max

#output dimensions of word and character embedding
hidden_size = 100
# char_vocab = 1218

#character CNN filters and width
n_filters = 100
filter_width = 3

tf.keras.backend.clear_session()
# model = bidaf_model(question_timesteps, context_timesteps, hidden_size, char_vocab, n_filters, filter_width)
model = bidaf_model(hidden_size,  n_filters, filter_width)

print(model.summary())

Model: "bidaf"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Ids (InputLayer)               [(None, 512)]        0           []                               
                                                                                                  
 mask (InputLayer)              [(None, 512)]        0           []                               
                                                                                                  
 Type (InputLayer)              [(None, 512)]        0           []                               
                                                                                                  
 bert (Bert)                    ((None, 52, 768),    109482241   ['Ids[0][0]',                    
                                 (None, 461, 768))                'mask[0][0]',               

In [ ]:
#https://medium.com/@aakashgoel12/how-to-add-user-defined-function-get-f1-score-in-keras-metrics-3013f979ce0d
def f1_score(y_true, y_pred):    #taken from old keras source code
    
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    
    return f1_val

## Loading pre trained model from disk for fine tuning

In [ ]:
model = tf.keras.models.load_model('/content/gdrive/MyDrive/Colab Notebooks/modelbase2')

In [ ]:
print(model.summary())

Model: "bidaf"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Ids (InputLayer)               [(None, 512)]        0           []                               
                                                                                                  
 mask (InputLayer)              [(None, 512)]        0           []                               
                                                                                                  
 Type (InputLayer)              [(None, 512)]        0           []                               
                                                                                                  
 bert (Bert)                    ((None, 52, 768),    109482241   ['Ids[0][0]',                    
                                 (None, 461, 768))                'mask[0][0]',               

### Prepare Data Pipeline

In [ ]:
#defining loss function and optimizer
loss_function = tf.keras.losses.CategoricalCrossentropy(reduction='auto')
optimizer = tf.keras.optimizers.Nadam(learning_rate=0.001)

In [ ]:
#batch size
BATCH_SIZE=16
##number of epochs
EPOCHS= 4

In [ ]:
##creating input dataset(train and test) using tf.data
train_inputs = tf.data.Dataset.from_tensor_slices((train_input_id,train_input_mask,train_input_type))
test_inputs = tf.data.Dataset.from_tensor_slices((test_input_id,test_input_mask,test_input_type))

In [ ]:
##creating output dataset(train and test) using tf.data
train_targets = tf.data.Dataset.from_tensor_slices((y_start_train, y_end_train))
test_targets = tf.data.Dataset.from_tensor_slices((y_start_test, y_end_test))

In [ ]:
#shuffling and split to batches
train_dataset = tf.data.Dataset.zip((train_inputs, train_targets)).shuffle(500).batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)
test_dataset = tf.data.Dataset.zip((test_inputs, test_targets)).shuffle(500).batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)

### Model Training

In [ ]:
#https://udai.gitbook.io/practical-ml/nn/training-and-debugging-of-nn
#steps to be performed in each training step
@tf.function
def train_step(input_vector, output_vector,loss_fn):
    with tf.GradientTape() as tape:
        #forward propagation
        # print(type(input_vector))
        # print(type(input_vector[0]))
        # print(input_vector[0].shape)
        # print(input_vector.shape)
        output_predicted = model(inputs=input_vector, training=True)

        # print(type(output_predicted))
        # print(type(input_vector[0]))
        # print(input_vector[0].shape)

        #loss
        loss_start = loss_function(output_vector[0], output_predicted[0])
        loss_end = loss_function(output_vector[1], output_predicted[1])
        loss_final = loss_start + loss_end
        # print('losses Calculated')
    #getting gradients
    gradients = tape.gradient(loss_final, model.trainable_variables)
    # print('Gradients Calculated')
    #applying gradients
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    # print('Gradients applied')
    return loss_start, loss_end, output_predicted, gradients

In [ ]:
#https://udai.gitbook.io/practical-ml/nn/training-and-debugging-of-nn
#steps to be performed in each validation step
@tf.function
def val_step(input_vector, output_vector, loss_fn):
    #getting output of validation data
    output_predicted = model(inputs=input_vector, training=False)
    #loss calculation
    loss_start = loss_function(output_vector[0], output_predicted[0])
    loss_end = loss_function(output_vector[1], output_predicted[1])
    return loss_start, loss_end, output_predicted

In [ ]:
#defining functions to compute the mean loss for each epoch
train_start_loss = tf.keras.metrics.Mean(name='train_start_loss')
train_end_loss = tf.keras.metrics.Mean(name='train_end_loss')
val_start_loss = tf.keras.metrics.Mean(name='val_start_loss')
val_end_loss = tf.keras.metrics.Mean(name='val_end_loss')
train_start_f1 = tf.keras.metrics.Mean(name="train_start_f1")
train_end_f1 = tf.keras.metrics.Mean(name="train_end_f1")
val_start_f1 = tf.keras.metrics.Mean(name="val_start_f1")
val_end_f1 = tf.keras.metrics.Mean(name="val_end_f1")

In [ ]:
#tensorboard file writers
wtrain = tf.summary.create_file_writer(logdir='logdir_train')
wval = tf.summary.create_file_writer(logdir='logdir_val')

In [ ]:
#to get the iteration number for recording logs
iters = math.ceil(64769/BATCH_SIZE) 
#for model checkpointing
best_loss=100

In [ ]:
for epoch in range(EPOCHS):
    
    #resetting the states of the loss and metrics
    train_start_loss.reset_states()
    train_end_loss.reset_states()
    val_start_loss.reset_states()
    val_end_loss.reset_states()
    train_start_f1.reset_states()
    train_end_f1.reset_states()
    val_start_f1.reset_states()
    val_end_f1.reset_states()
    
    ##counter for train loop iteration
    counter = 0
    
    #ietrating over train data batch by batch
    for text_seq, label_seq in tqdm(iterable=train_dataset, total=len(list(train_dataset))):
        #train step
        try:
            loss_start_, loss_end_, pred_out, gradients = train_step(text_seq, label_seq, loss_function)
        except:
            continue
        #adding loss to train loss
        train_start_loss(loss_start_)
        train_end_loss(loss_end_)
        #counting the step number
        temp_step = epoch*iters+counter
        counter = counter + 1
        
        #calculating f1 for batch
        f1_start = f1_score(label_seq[0], pred_out[0])
        f1_end = f1_score(label_seq[1], pred_out[1])
        train_start_f1(f1_start)
        train_end_f1(f1_end)
        
        # ##tensorboard 
        # with tf.name_scope('per_step_training'):
        #     with wtrain.as_default():
        #         tf.summary.scalar("start_loss", loss_start_, step=temp_step)
        #         tf.summary.scalar("end_loss", loss_end_, step=temp_step)
        #         tf.summary.scalar('f1_start', f1_start, step=temp_step)
        #         tf.summary.scalar('f1_end', f1_end, step=temp_step)
        # with tf.name_scope("per_batch_gradients"):
        #     with wtrain.as_default():
        #         for i in range(len(model.trainable_variables)):
        #             name_temp = model.trainable_variables[i].name
        #             tf.summary.histogram(name_temp, gradients[i], step=temp_step)
    
    
    #validation data
    for text_seq_val, label_seq_val in test_dataset:
        #getting val output
        try:
            loss_val_start, loss_val_end, pred_out_val = val_step(text_seq_val, label_seq_val, loss_function)
        except:
            continue
        
        val_start_loss(loss_val_start)
        val_end_loss(loss_val_end)
        
        #calculating metric
        f1_start_val = f1_score(label_seq_val[0], pred_out_val[0])
        f1_end_val = f1_score(label_seq_val[1], pred_out_val[1])
        val_start_f1(f1_start_val)
        val_end_f1(f1_end_val)
    
   
    #printing
    template = '''Epoch {}, Train Start Loss: {:0.6f}, Start F1 Score: {:0.5f}, Train End Loss: {:0.6f}, End F1 Score: {:0.5f},
    #Val Start Loss: {:0.6f}, Val Start F1 Score: {:0.5f}, Val End Loss: {:0.6f}, Val End F1 Score: {:0.5f}'''

    print(template.format(epoch+1, train_start_loss.result(), train_start_f1.result(), 
                          train_end_loss.result(), train_end_f1.result(),
                          val_start_loss.result(), val_start_f1.result(),
                          val_end_loss.result(), val_end_f1.result()))


    # if (val_start_loss.result()+val_end_loss.result())<best_loss:
    #   model.save("drive/My Drive/Colab Notebooks/new_model/model")
    #   best_loss=(val_start_loss.result()+val_end_loss.result())
    
    # #tensorboard
    # with tf.name_scope("per_epoch_loss_metric"):
    #     with wtrain.as_default():
    #         tf.summary.scalar("start_loss", train_start_loss.result().numpy(), step=epoch)
    #         tf.summary.scalar("end_loss", train_end_loss.result().numpy(), step=epoch)
    #         tf.summary.scalar('start_f1', train_start_f1.result().numpy(), step=epoch)
    #         tf.summary.scalar('end_f1', train_end_f1.result().numpy(), step=epoch)
    #     with wval.as_default():
    #         tf.summary.scalar("start_loss", val_start_loss.result().numpy(), step=epoch)
    #         tf.summary.scalar("end_loss", val_end_loss.result().numpy(), step=epoch)
    #         tf.summary.scalar('start_f1', val_start_f1.result().numpy(), step=epoch)
    #         tf.summary.scalar('end_f1', val_end_f1.result().numpy(), step=epoch)

100%|██████████| 150/150 [07:14<00:00,  2.89s/it]


Epoch 1, Train Start Loss: 0.988486, Start F1 Score: 0.86356, Train End Loss: 1.081917, End F1 Score: 0.84867,
    #Val Start Loss: 0.000169, Val Start F1 Score: 1.00000, Val End Loss: 0.000086, Val End F1 Score: 1.00000


  9%|▊         | 13/150 [00:35<06:15,  2.74s/it]

In [ ]:
#saving the model to disk
model.save("/content/gdrive/MyDrive/Colab Notebooks/modelbasemoddata")

In [ ]:
#saving tensorboard logs
%cp logdir_train/*.v2 "drive/My Drive/Colab Notebooks/logdir/train/"
%cp logdir_val/*.v2 "drive/My Drive/Colab Notebooks/logdir/val/"

In [ ]:
# def print_predictions(data_point):
#   """Function that takes record numbers as input and predicts the answer for that record"""

tf.expand_dims(train_input_type[0],axis=0).shape

start,end = model([tf.expand_dims(train_input_id[0],axis=0),tf.expand_dims(train_input_mask[0],axis=0),tf.expand_dims(train_input_type[0],axis=0)],training=True)
    
#   print("Question:")
#   for i in test_question_word_padded[data_point]:
#     if i==0:
#       break
#     else:
#       print(list(word_tokenizer.keys())[list(word_tokenizer.values()).index(i)], end = ' ')
#   print("\nContext:")
#   for i in test_context_word_padded[data_point]:
#     if i==0:
#       break
#     else:
#       print(list(word_tokenizer.keys())[list(word_tokenizer.values()).index(i)], end = ' ')
#   print("\nPredicted Answer:")
#   start, end = model.predict([test_question_word_padded[data_point:data_point+1], test_context_word_padded[data_point:data_point+1], test_question_char_padded[15:16], test_context_char_padded[15:16]])
#   for i in range(start.argmax(), end.argmax()+1):
#     print(list(word_tokenizer.keys())[list(word_tokenizer.values()).index(test_context_word_padded[data_point][i])], end=' ')
#   print("\n")

InvalidArgumentError: ignored